In [40]:
def test_with_yield():
    for i in range(10):
        yield lambda x: x+i
a = test_with_yield()
next(a)

<function __main__.test_with_yield.<locals>.<lambda>(x)>

In [41]:
import numpy as np
np.fromiter('abcdefg', (np.unicode,1))

array(['a', 'b', 'c', 'd', 'e', 'f', 'g'], dtype='<U1')

In [6]:
import itertools
# len(list(itertools.permutations([1,2,3, 6, 6,1,4], 4)))
len(list(itertools.permutations([1]*80, 4)))

37957920

In [1]:
for p in find_permutations([[5,1,2], [5,1,6], [5,1,7]], full_set):
    print(p)

NameError: name 'find_permutations' is not defined

In [4]:
any([False, False, True])

True

In [7]:
any([False, False, False])

False

In [13]:
import itertools
list(itertools.permutations([1,2,3, 6, 7, 8], 1))
# len(list(itertools.permutations([1]*80, 4)))

[(1,), (2,), (3,), (6,), (7,), (8,)]

In [19]:
import itertools
x = itertools.permutations([1]*80, 5)
print(next(x))
print(next(x))
print(next(x))

(1, 1, 1, 1, 1)
(1, 1, 1, 1, 1)
(1, 1, 1, 1, 1)


In [34]:
import itertools
x = itertools.permutations([1]*2, 1)

print(next(x))
print(next(x))
    
try:
    print(next(x))
except StopIteration:
    print('asd')

# a = []
# a.append(next(x))
# a is None

(1,)
(1,)
asd


In [37]:
import itertools
x = itertools.permutations([1]*2, 1)

while True:
    try:
        print(next(x))
    except StopIteration:
        print('done')
        break

(1,)
(1,)
done


In [41]:
import itertools
x = itertools.permutations([1]*2, 1)

while True:
    out = next(x, None)
    print(out)
    if out is None:
        break


(1,)
(1,)
None


In [37]:
# https://github.com/numba/numba/issues/3599

from numba import njit
from numba.extending import overload
import itertools

@overload(itertools.permutations)
def permutations(A, k):
    def impl(A, k):
        # Not actually a permutation implementation,
        # Just a demonstration of generator support.
        for i in range(k):
            yield A[i:] + A[:i]
    return impl

@njit
def test(A, k):
    for i in itertools.permutations(A, k):
        print(i)


r = test([1, 2, 3, 4,3], k=6)
# print(next(r))


[1, 2, 3, 4, 3]
[2, 3, 4, 3, 1]
[3, 4, 3, 1, 2]
[4, 3, 1, 2, 3]
[3, 1, 2, 3, 4]
[1, 2, 3, 4, 3]


In [38]:
def permutations(elements):
    if len(elements) <=1:
        yield elements
    else:
        for perm in permutations(elements[1:]):
            for i in range(len(elements)):
                # nb elements[0:1] works in both string and list contexts
                yield perm[:i] + elements[0:1] + perm[i:]
                
x = permutations([1,2,4,3])

print(next(x))
print(next(x))

[1, 2, 4, 3]
[2, 1, 4, 3]


In [39]:
def permutations(iterable, r=None):
    # permutations('ABCD', 2) --> AB AC AD BA BC BD CA CB CD DA DB DC
    # permutations(range(3)) --> 012 021 102 120 201 210
    pool = tuple(iterable)
    n = len(pool)
    r = n if r is None else r
    if r > n:
        return
    indices = list(range(n))
    cycles = list(range(n, n-r, -1))
    yield tuple(pool[i] for i in indices[:r])
    while n:
        for i in reversed(range(r)):
            cycles[i] -= 1
            if cycles[i] == 0:
                indices[i:] = indices[i+1:] + indices[i:i+1]
                cycles[i] = n - i
            else:
                j = cycles[i]
                indices[i], indices[-j] = indices[-j], indices[i]
                yield tuple(pool[i] for i in indices[:r])
                break
        else:
            return
x = permutations([1,2,4,3], 2)

print(next(x))
print(next(x))
print(next(x))
print(next(x))


(1, 2)
(1, 4)
(1, 3)
(2, 1)


# JIT

In [13]:
import numba as nb
import numpy as np
import math

@nb.jit(nopython=True)
def num_permutations_no_repetition(n, r):
    # return math.factorial(n) / math.factorial(n-r)
    return np.rint(math.e ** (math.lgamma(n + 1) - math.lgamma(n - r + 1)))


@nb.jit(nopython=True)
def permutations(pool, r=None):
    n = pool.shape[0]
    r = n if r is None else r
    if r > n:
        return
    indices = np.arange(n)
    cycles = np.arange(n, n-r, -1)
    ret = np.empty(r, dtype=np.uint32)
    for i in range(r):
        ret[i] = pool[indices[:r][i]]
    yield ret
    while n:
        for i in range(r - 1, -1, -1):
            cycles[i] -= 1
            if cycles[i] == 0:
                indices[i:] = np.concatenate((indices[i+1:], indices[i:i+1]))
                cycles[i] = n - i
            else:
                j = cycles[i]
                indices[i], indices[-j] = indices[-j], indices[i]
                # retPre = [pool[i] for i in indices[:r]]
                ret = np.empty(r, dtype=np.uint32)
                for i in range(r):
                    ret[i] = pool[indices[:r][i]]
                yield ret
                break
        else:
            return


@nb.njit(fastmath=True)
def pair_in_mat(pair, mat):
    for i in nb.prange(mat.shape[0]):
        if np.all(pair == mat[i]):
            return True
        if np.all(np.array([pair[1], pair[0]]) == mat[i]):
            return True
    return False


@nb.njit(fastmath=True, parallel=True)
# @nb.njit(parallel=True)
def combos_in_valid_pairs(combo_check, valid_pairs):  # if 2d array
    combo_results = np.zeros(combo_check.shape[0], dtype=np.bool_)

    for i in nb.prange(combo_check.shape[0]):
        valid = pair_in_mat(combo_check[i], valid_pairs)
        # if not valid:
        #     return False  # isn't supported by parallelization
        combo_results[i] = valid

    return np.all(combo_results)


# @nb.jit(nopython=True, fastmath=True)
def create_perms_inner(currs_considered, exg, num_currs, valid_pairs, end_at_same):
    ret = np.zeros((1000000, num_currs), dtype=np.uint32)

    # perms_pre = [['BNB', 'PAX', 'BTC'], ['ETH', '2342', 'BTC'],
    #               ['ETH', 'TUSD', 'USDT'], ['ETH', 'USDC', 'USDT']]  # for testing
    # for row in range(len(perms_pre)):
    #     for col in range(len(perms_pre[0])):
    #         perms_pre[row][col] = numify(perms_pre[row][col])  # for testing
    # perms = iter(perms_pre)  # for testing

    perms = permutations(currs_considered, num_currs - 1)

    if end_at_same:
        # num_perms = len(perms_pre)  # for testing
        num_perms = num_permutations_no_repetition(len(currs_considered), num_currs - 1)
    else:
        # num_perms = len(perms_pre)  # for testing
        num_perms = num_permutations_no_repetition(len(currs_considered), num_currs)

    num_perms = int(num_perms)
    # while True:
    for n in range(num_perms):
        perm = next(perms)
        if perm is None:
            break

        perm = np.append(perm, perm[0])
        combos_check = np.empty((perm.size - 1, 2), dtype=np.uint32)
        for i in range(perm.size - 1):
            combos_check[i, 0] = perm[i]
            combos_check[i, 1] = perm[i + 1]

        valid = combos_in_valid_pairs(combos_check, valid_pairs)
        if valid:
            ret[n, :] = perm



            a=1
    # [[4345410    4277313    4346947    4345410], [4345410    4277313 1112888132    4345410],
    #  [4345410    4277313    4543560    4345410], [4345410    4277313 1414878020    4345410],
    #  [4345410    4277313 1431520323    4345410], [4345410    4277313 1431520340    4345410]]

    # np.savetxt("foo.csv", valid_pairs, delimiter=",")

    return ret

create_perms_inner(np.array([1, 2, 3, 4, 5, 6]), 'asd', 3, np.array([[3, 2],[3, 4], [1, 2]]), True)

array([[1, 2, 1],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=uint32)

In [11]:
np.array([[3, 2],[3, 4], [1, 2]])

array([[3, 2],
       [3, 4],
       [1, 2]])